# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab
import numpy as np

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:315: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:120: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
[WARNING] Unable to write current GraphLab Create license to /home/kuntal/.graphlab/config. Ensure that this user account                         has write permiss

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Find what features had non-zero weight.

In [6]:
model_all.get("coefficients").print_rows(num_rows=18, num_columns=3)
model_all['coefficients']['value'].nnz()

+------------------+-------+---------------+-----+
|       name       | index |     value     | ... |
+------------------+-------+---------------+-----+
|   (intercept)    |  None |  274873.05595 | ... |
|     bedrooms     |  None |      0.0      | ... |
| bedrooms_square  |  None |      0.0      | ... |
|    bathrooms     |  None | 8468.53108691 | ... |
|   sqft_living    |  None | 24.4207209824 | ... |
| sqft_living_sqrt |  None | 350.060553386 | ... |
|     sqft_lot     |  None |      0.0      | ... |
|  sqft_lot_sqrt   |  None |      0.0      | ... |
|      floors      |  None |      0.0      | ... |
|  floors_square   |  None |      0.0      | ... |
|    waterfront    |  None |      0.0      | ... |
|       view       |  None |      0.0      | ... |
|    condition     |  None |      0.0      | ... |
|      grade       |  None | 842.068034898 | ... |
|    sqft_above    |  None | 20.0247224171 | ... |
|  sqft_basement   |  None |      0.0      | ... |
|     yr_built     |  None |   

6

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [8]:
for l1_penalty in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,validation_set=None, verbose = False,l2_penalty=0., l1_penalty=l1_penalty)
    prediction=model.predict(validation)
    RSS=np.sum(np.square(prediction - validation['price']))
    print (l1_penalty , RSS)
    

(10.0, 625766285142460.5)
(31.622776601683793, 625766285362394.0)
(100.0, 625766286057885.25)
(316.22776601683796, 625766288257224.88)
(1000.0, 625766295212186.12)
(3162.2776601683795, 625766317206080.62)
(10000.0, 625766386760658.25)
(31622.776601683792, 625766606749278.75)
(100000.0, 625767302791635.12)
(316227.76601683791, 625769507643885.88)
(1000000.0, 625776517727024.5)
(3162277.6601683795, 625799062845466.62)
(10000000.0, 625883719085425.38)


In [9]:
model_10 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0, l1_penalty=10)
model_10['coefficients']['value'].nnz()

18

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [10]:
prediction=model_10.predict(testing)

RSS=np.sum(np.square(prediction - testing['price']))
print  RSS

1.56983602382e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [11]:
model_10['coefficients']['value'].nnz()

18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [12]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [13]:
l1_penalty_values = np.logspace(1, 8, num=20)



Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [17]:
nonzeros=[]
for l1_penalty in np.logspace(8, 10, num=20):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,validation_set=None, verbose = False,l2_penalty=0., l1_penalty=l1_penalty)
    nnz=model['coefficients']['value'].nnz()
    nonzeros.append(nnz)
    prediction=model.predict(validation)
    RSS=np.sum(np.square(prediction - validation['price']))
    print (l1_penalty , nnz,  RSS)
print nonzeros

(100000000.0, 18, 627492659875075.38)
(127427498.57031322, 18, 628210516770747.25)
(162377673.91887242, 18, 629176689541061.5)
(206913808.11147901, 18, 630650082718674.62)
(263665089.87303555, 17, 632940229286984.25)
(335981828.62837881, 17, 636268140229529.5)
(428133239.8719396, 17, 641261198311187.75)
(545559478.11685145, 17, 648983455376337.25)
(695192796.17755914, 17, 660962217696068.62)
(885866790.41008317, 16, 677261520727858.75)
(1128837891.6846883, 15, 701046815867076.38)
(1438449888.2876658, 15, 737850622829048.25)
(1832980710.8324375, 13, 796163109639833.12)
(2335721469.0901213, 12, 869018172893693.5)
(2976351441.6313133, 10, 966925692362085.38)
(3792690190.7322536, 6, 1081867592324112.5)
(4832930238.5717525, 5, 1244927360324958.8)
(6158482110.6602545, 3, 1384161490235112.2)
(7847599703.5146227, 1, 1230794720455646.2)
(10000000000.0, 1, 1229157160638600.2)
[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [18]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [19]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [20]:
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,validation_set=None, verbose = False,l2_penalty=0., l1_penalty=l1_penalty)
    nnz=model['coefficients']['value'].nnz()
    prediction=model.predict(validation)
    RSS=np.sum(np.square(prediction - validation['price']))
    print (l1_penalty , nnz ,  RSS)

(2976351441.6300001, 10, 966925692362085.38)
(3019316638.9510527, 10, 974019450084554.88)
(3062281836.2721052, 10, 981188367942454.12)
(3105247033.5931578, 10, 989328342459473.75)
(3148212230.9142108, 10, 998783211265890.0)
(3191177428.2352633, 10, 1008477167020093.8)
(3234142625.5563159, 10, 1018298780553819.8)
(3277107822.8773685, 10, 1028247992205977.1)
(3320073020.198421, 8, 1034616909232827.2)
(3363038217.5194736, 8, 1038554735941040.5)
(3406003414.8405266, 8, 1043237237871701.8)
(3448968612.1615791, 7, 1046937488751711.0)
(3491933809.4826317, 7, 1051147625612862.5)
(3534899006.8036842, 7, 1055992735342999.8)
(3577864204.1247368, 7, 1060799531763287.5)
(3620829401.4457893, 6, 1065707689498230.1)
(3663794598.7668419, 6, 1069464335425585.0)
(3706759796.0878949, 6, 1073504549585599.5)
(3749724993.4089475, 6, 1077632775581415.5)
(3792690190.73, 6, 1081867592324112.5)


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [21]:
l1_penalty=3448968612.1615791
model = graphlab.linear_regression.create(training, target='price', features=all_features,validation_set=None, verbose = False,l2_penalty=0., l1_penalty=l1_penalty)

print model['coefficients']

print model['coefficients']['value'].nnz()


+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
+------------------+-------+---------------+--------+
[18 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
7
